Copyright 2023 Borislav Rumenov Varbanov

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Simple 3D Cube Rendering from Side Perspective
This notebook demonstrates a minimal 3D pipeline that rotates and projects
a cube before rasterizing it with Matplotlib.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x_res, y_res = 800, 640

cube_vertices = np.array([
    [-1,-1,-1],
    [ 1,-1,-1],
    [ 1, 1,-1],
    [-1, 1,-1],
    [-1,-1, 1],
    [ 1,-1, 1],
    [ 1, 1, 1],
    [-1, 1, 1]
])

faces = [
    [0,1,2,3],
    [4,5,6,7],
    [0,1,5,4],
    [2,3,7,6],
    [1,2,6,5],
    [0,3,7,4]
]

ang_y = np.radians(45)
ang_x = np.radians(25)
R_y = np.array([
    [np.cos(ang_y),0,np.sin(ang_y)],
    [0,1,0],
    [-np.sin(ang_y),0,np.cos(ang_y)]
])
R_x = np.array([
    [1,0,0],
    [0,np.cos(ang_x),-np.sin(ang_x)],
    [0,np.sin(ang_x),np.cos(ang_x)]
])

rotated = cube_vertices @ R_y.T @ R_x.T

def project(v, d=5):
    z = v[2] + d
    return np.array([v[0]/z, v[1]/z])

projected = np.array([project(v) for v in rotated])
projected[:,0] = (projected[:,0] + 1) * x_res/2
projected[:,1] = (1 - projected[:,1]) * y_res/2

fig, ax = plt.subplots()
for face in faces:
    poly = projected[face]
    ax.fill(poly[:,0], poly[:,1], edgecolor='black', alpha=0.5)
ax.set_xlim(0, x_res)
ax.set_ylim(0, y_res)
ax.set_aspect('equal')
ax.axis('off')
plt.savefig('cube_side_perspective.png')
plt.show()
